In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [ ]:
import numpy as np
import matplotlib.pylab as plt
import itertools
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score

# import keras
# from keras import backend as K
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, model_from_json
# from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D, SeparableConv2D


In [ ]:
X = np.load('/content/drive/MyDrive/b_cancer/X.npy') # images
Y = np.load('/content/drive/MyDrive/b_cancer/Y.npy') # labels associated to images (0 = no IDC, 1 = IDC)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
# 0과 1 사이로 스케일 조정

X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
y_train = to_categorical(y_train, num_classes = 2)
y_test = to_categorical(y_test, num_classes = 2)

In [ ]:
from keras.applications.resnet_v2 import ResNet50V2

img_shape = (50, 50, 3)

with tf.device('/device:GPU:0'):
  base_model = tf.keras.applications.resnet_v2.ResNet50V2(input_shape=img_shape, include_top=False, classes=2)

  model = Sequential(base_model)
  model.add(Flatten())
  model.add(Dense(256))
  model.add(Activation("relu"))
  model.add(BatchNormalization())
 
  # softmax classifier
  model.add(Dense(2))
  model.add(Activation("softmax"))

94683136/94668760 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2, 2, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                        

In [ ]:
with tf.device('/device:GPU:0'):
  NUM_EPOCHS = 40
  INIT_LR = 1e-2
  BS = 128
  opt = Adam(learning_rate=INIT_LR)
  model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

  H = model.fit(X_train, y_train,
                batch_size = BS,
                verbose = 1,
                validation_data = (X_test, y_test),
                epochs = NUM_EPOCHS)

Epoch 1/40
31/31 [==============================] - 26s 226ms/step - loss: 0.9683 - accuracy: 0.5085 - val_loss: 17927711037063168.0000 - val_accuracy: 0.4865
Epoch 2/40
31/31 [==============================] - 4s 128ms/step - loss: 0.7575 - accuracy: 0.5033 - val_loss: 570840896.0000 - val_accuracy: 0.4865
Epoch 3/40
31/31 [==============================] - 4s 121ms/step - loss: 0.7251 - accuracy: 0.5046 - val_loss: 59832.8438 - val_accuracy: 0.5135
Epoch 4/40
31/31 [==============================] - 4s 122ms/step - loss: 0.7296 - accuracy: 0.5054 - val_loss: 3.1229 - val_accuracy: 0.4853
Epoch 5/40
31/31 [==============================] - 4s 130ms/step - loss: 0.7264 - accuracy: 0.5075 - val_loss: 2.6594 - val_accuracy: 0.5135
Epoch 6/40
31/31 [==============================] - 4s 124ms/step - loss: 0.7362 - accuracy: 0.5160 - val_loss: 0.9218 - val_accuracy: 0.5153
Epoch 7/40
31/31 [==============================] - 4s 126ms/step - loss: 0.7459 - accuracy: 0.5121 - val_loss: 1.7229 